[IRIS]

Feature : 3개 >>>
Target : 1개 >>>


[IRIS]

Feature : 4개 >>>
Target : 1개 >>>
class : 2개 >>>


[IRIS]

Feature : 4개 >>>
Target : 1개 >>>

softmax 가 어떻게 값은내놓길레? 다중분류에서 쓰걸까?

#### DNN 기반 회귀 모델 구현
- 데이터셋 : iris.csv
- 피쳐/라벨 : 3개 sepal.length, sepal.width, petal.length
- 타겟/라벨 : 1개 petal.width
- 학습 방법 : 지도 학습 > 회귀
- 학습 알고리즘 : 인공신경망(ANN) =>> 심층신경막(입력층, 은닉층, 출력층 있는거) MLP , DNN : 은닉층이 많은 구성
- 프레임 워크 : Pytorch

[1] 모듈 로딩 및 데이터 준비 <hr>

In [1]:
# 모듈 로딩
# 1. 모델관련
import torch                                    # 텐서 및 수치 계산 함수 관련 모듈
import torch.nn as nn                           # 인공신경망 관련 모듈
import torch.nn.functional as F                 # 손실, 거리 등 함수 관련 모듈

# 2. 데이터 셋 관련                    
from torch.utils.data import DataLoader, Dataset

# 3. 최적화에 관련
import torch.optim as optim    # 최적화 기법 관련 모듈

# 4. 모델 평
from torchmetrics.regression import R2Score, MeanSquaredError

# 5. 모델의 구조를 보는 모듈
from torchinfo import summary                   # 모델 정보 관련 모듈

# 6. Data 관련
import pandas as pd                 # 데이터 파일 분석 관련 모듈
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [2]:
# 활용 패키지 버전 체크 ==> 사용자 정의 함수로 구현하세요~~
print(f'Pytroch v.{torch.__version__}')
print(f'Pandas v.{pd.__version__}')

Pytroch v.2.4.1
Pandas v.2.0.3


In [3]:
# 데이터 로딩
DATA_file ='../data/iris.csv'

# CSV >> DF
irisDF = pd.read_csv(DATA_file, usecols=[0,1,2,3])

# 확인
irisDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2


[2] 모델 클래스 설계 및 정의 <hr>
- 클래스 목적 : iris 데이터를 학습 및 추론 목적
- 클래스 이름 : IrisRegModel
- 부모 클래스 : nn.Module
- 매개 변수 : 층별 입출력 개수 고정하기 때문에 필요 없음
- 속성 필드 : 
- 기능 역할(필수 메서드)     
    + _ _init_ _() - 모델 구조 설정     
    + forward() - 순방향 학습 <== 오버라이딩(overriding) >> 상속받을 때만 가능
- 클래스 구조
    * 입력층 - 입력  3개 (피쳐 개수)  >  출력 10개 (퍼셉트론 / 뉴런 10개 존재)
    * 은닉층 - 입력 10개             >  출력 30개 (퍼셉트론 / 뉴런 30개 존재)
    * 출력층 - 입력 30개             >  출력  1개 (타겟, [너비값(petal.width)])
- 활성화함수
    * 클래스 형태 ==> nn.MESLose, nn.ReLU ==> _ _init_ _() 메서드
    * 함수 형태 ==> torch.nn.functional 아래에 ==> forward() 메서드

In [4]:
class IrisRegModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer = nn.Linear(3,10)
        self.hd_layer = nn.Linear(10,30)
        self.out_layer = nn.Linear(30,1)

    # 순방향 학습 진행 메서드
    def forward(self, input_data):

        # 입력층 
        y = self.in_layer(input_data)   # fi1W11+f12W12+f13W13+b, ... , fi101W101+f102W102+fi103W103+b
        y = F.relu(y)                   # relu => y값의 범위 : 0 <= y

        # 은닉층 : 10개의 숫자 값(y >= 0)
        y=self.hd_layer(y)              # f21W11+f22W12...+f210W210+b, ... , f230W201+...+f230W210+b
        y = F.relu(y)                   # relu => y 값의 범위 : 0 <= y

        # 출력층 : 30개의 숫자 값(y >= 0) >> 회귀이므로 바로 반환(return)
        return self.out_layer(y)        # f31w31+......+f330w330 + b

In [5]:
# 모델 인스턴스 생성
model = IrisRegModel() # >>> 지금 이거 본건 모델이 잘나왔냐 확인차
print(model)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


In [6]:
# 모델 사용 메로리 정보 확인
summary(model, input_size=(100000,3)) # 데이터양, 피쳐개수

Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [100000, 1]               --
├─Linear: 1-1                            [100000, 10]              40
├─Linear: 1-2                            [100000, 30]              330
├─Linear: 1-3                            [100000, 1]               31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 40.10
Input size (MB): 1.20
Forward/backward pass size (MB): 32.80
Params size (MB): 0.00
Estimated Total Size (MB): 34.00

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피쳐 개수 : 3개
- 타겟 개수 : 1개
- 클래스 이름 : IrisDataset
- 부모클래스 : utils.data.Dataset
- 속성_필드 : featuresDF, targetDF, n_rows, n_features
- 필수메서드
    * __init__(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    * __len__(self) : 데이터의 개수 반환
    * __getitme__(self, index) : 특정 인덱스의 피쳐와 타겟 반환

In [7]:
class IrisDataset(Dataset):
    def __init__(self, featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]
    
    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
    
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values) # values하는 이유 >> array를 하기위해
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)

        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [8]:
# 데이터셋 인스턴스 생성 

# DF에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]] # 2D (150,3)
targetDF = irisDF[irisDF.columns[-1:]]  # 2D (150,1)

# 커스텀데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF, targetDF)

[4] 학습 준비 <hr>
- 학습 횟수 : EPOCH     <<< 처음 ~ 끝까지 공부하는 단위
- 배치 크기 : BATCH_SIZE    <<< 한번에 학급할 데이터셋 양
- 위치 지정 : DEVICE    <<< 텐서 저장 및 실행 위치(GPU/CPU)
- 학습률 : LR   <<< 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정, 0.001 ~ 0.1 사이를 많이 줌(값이 작을수록 촘촘하게 이동) >> 하이퍼 파라미터

In [9]:
## 학습 진행 관련 설정
EPOCH = 1
BATCH_SIZE = 10
BATCH_CNT = irisDF.shape[0]//BATCH_SIZE
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

print(f'BATCH_CNT : {BATCH_CNT}')

BATCH_CNT : 15


- 인스턴스/객체 : 모델, 데이터셋, 최적화 (, 손실함수 , 성능 지표)

In [10]:
# 모델 인스턴스
model = IrisRegModel()

# 데이터셋 인스턴스
X_train, X_test, y_train, y_test = train_test_split(featureDF,targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)
print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{type(X_train)} {type(X_test)} {type(X_val)}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')
print(f'{type(y_train)} {type(y_test)} {type(y_val)}')

trainDS = IrisDataset(X_train, y_train)
testDS = IrisDataset(X_test, y_test)
valDS = IrisDataset(X_val, y_val)

# 데이터로더 인스턴스
trainDL = DataLoader(trainDS, batch_size=BATCH_SIZE)

(84, 3) (38, 3) (28, 3)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
(84, 1) (38, 1) (28, 1)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [11]:
## [테스트] 데이터로더와 데이터셋 체크
for feature, target in trainDL:
    print(feature.shape, target.shape)

torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([4, 3]) torch.Size([4, 1])


In [12]:
# 최적화 인스턴스 => W,b텐서 즉, model.parameters() 전달
#                >> W,b 업데이트 시키는이유 >>> 오차를 줄여 최적의 모델을 찾기위해  
optimizer = optim.Adam(model.parameters(), lr=LR)

# 소실함수 인스턴스 => 회귀, MSE, MAE, RMSE ....
reqLoss = nn.MSELoss()

[5] 학습진행 <hr>

In [13]:
## 학습의 효과 확인, 손실값과 성능평가값 저장 필요 , 검증기능을 
LOSS_HISTORY, SCORE_HISTORY = [[], []], [[], []]
CNT=irisDS.n_rows/BATCH_SIZE

for epoch in range(EPOCH):
    # 학습 모드로 모델 설정
    model.train()
    
    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total = 0, 0
    for featureTS, targetTS in trainDL:

        # 학습 진행
        pre_y = model(featureTS)

        # 손실 계산
        loss = reqLoss(pre_y, targetTS)
        loss_total += loss.item()

        # 성능 평가 계산
        score = R2Score()(pre_y, targetTS) 
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크 당 검증 기능
    # 모델 검증 모드 설정 >> 검증이기에 업데이트 불필요
    model.eval() 
    with torch.no_grad():
        
        # 검증 데이터셋
        val_featrueTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)

        # 추론 / 평가 
        pre_val = model(val_featrueTS)
        
        # 손실
        loss_val = reqLoss(pre_val, val_targetTS)
        
        # 성능평가
        score_val = R2Score()(pre_val, val_targetTS)

    
    # 에포크 당 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/BATCH_CNT)
    SCORE_HISTORY[0].append(score_total/BATCH_CNT)

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)


In [14]:
val_featrueTS=torch.FloatTensor(valDS.featureDF.values)
val_targetTS=torch.FloatTensor(valDS.targetDF.values)

val_featrueTS.shape, val_targetTS.shape

(torch.Size([28, 3]), torch.Size([28, 1]))

In [15]:
print(f'CNT => {CNT}')
print(f'LOSS_HISTORY => {LOSS_HISTORY}')
print(f'SCORE_HISTORY => {SCORE_HISTORY}')


CNT => 15.0
LOSS_HISTORY => [[0.3895504355430603], [tensor(0.4689)]]
SCORE_HISTORY => [[-0.23987299998601277], [tensor(0.2736)]]
